In [ ]:
from datasets import load_dataset

dataset = load_dataset('cnn_dailymail', '3.0.0')


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/15.6k [00:00<?, ?B/s]

train-00000-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00001-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00002-of-00003.parquet:   0%|          | 0.00/259M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/34.7M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/30.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [ ]:
train_subset = dataset['train'].select(range(1000))
val_subset = dataset['validation'].select(range(100))
test_subset = dataset['test'].select(range(100))

In [ ]:
from transformers import T5Tokenizer

tokenizer = T5Tokenizer.from_pretrained('t5-small')


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [ ]:
def preprocess_data(examples):
    inputs = ['summarize: ' + doc for doc in examples['article']]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding='max_length')


    labels = tokenizer(examples['highlights'], max_length=150, truncation=True, padding='max_length')
    model_inputs['labels'] = labels['input_ids']
    return model_inputs


tokenized_train = train_subset.map(preprocess_data, batched=True)
tokenized_val = val_subset.map(preprocess_data, batched=True)
tokenized_test = test_subset.map(preprocess_data, batched=True)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [ ]:
from transformers import TrainingArguments


training_args = TrainingArguments(
    output_dir='./results',
    per_device_train_batch_size=4,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    logging_dir='./logs',
    eval_steps=500,
    save_steps=500,
)


In [ ]:
from transformers import T5ForConditionalGeneration
model = T5ForConditionalGeneration.from_pretrained('t5-small')


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
from transformers import Trainer


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
)


In [ ]:

trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: tanaziqubal (tanaziqubal-psg-college-of-technology) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss
500,1.302800


TrainOutput(global_step=750, training_loss=1.1654424845377604, metrics={'train_runtime': 167.5644, 'train_samples_per_second': 17.904, 'train_steps_per_second': 4.476, 'total_flos': 406025404416000.0, 'train_loss': 1.1654424845377604, 'epoch': 3.0})

In [ ]:

trainer.evaluate(tokenized_test)


{'eval_loss': 0.7036249041557312,
 'eval_runtime': 1.5745,
 'eval_samples_per_second': 63.512,
 'eval_steps_per_second': 8.257,
 'epoch': 3.0}

In [ ]:

model.save_pretrained('./my_t5_model')
tokenizer.save_pretrained('./my_t5_model')


('./my_t5_model/tokenizer_config.json',
 './my_t5_model/special_tokens_map.json',
 './my_t5_model/spiece.model',
 './my_t5_model/added_tokens.json')

In [ ]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=bd2fb9cfa1f942bd5ea723c21c4b9aa243537325f8b626a4c9c3d91711404e07
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score


In [ ]:
from rouge_score import rouge_scorer
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration


model = T5ForConditionalGeneration.from_pretrained('./my_t5_model')
tokenizer = T5Tokenizer.from_pretrained('./my_t5_model')
model.eval()


scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

predictions = []
references = []

for example in test_subset:
    input_text = example['article']
    reference_summary = example['highlights']


    input_ids = tokenizer.encode("summarize: " + input_text, return_tensors="pt", truncation=True, max_length=512)

    with torch.no_grad():
        output_ids = model.generate(input_ids, max_length=150, num_beams=4, early_stopping=True)
    predicted_summary = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    predictions.append(predicted_summary)
    references.append(reference_summary)


rouge_results = {"rouge1": [], "rouge2": [], "rougeL": []}

for prediction, reference in zip(predictions, references):
    scores = scorer.score(reference, prediction)
    for key in rouge_results:
        rouge_results[key].append(scores[key].fmeasure)


for key, values in rouge_results.items():
    avg_f1 = sum(values) / len(values)
    print(f"{key.upper()} F1 score: {avg_f1:.4f}")


ROUGE1 F1 score: 0.3303
ROUGE2 F1 score: 0.1370
ROUGEL F1 score: 0.2452


In [ ]:
!zip -r my_t5_model.zip my_t5_model/

  adding: my_t5_model/ (stored 0%)
  adding: my_t5_model/config.json (deflated 62%)
  adding: my_t5_model/generation_config.json (deflated 29%)
  adding: my_t5_model/added_tokens.json (deflated 83%)
  adding: my_t5_model/special_tokens_map.json (deflated 85%)
  adding: my_t5_model/tokenizer_config.json (deflated 94%)
  adding: my_t5_model/spiece.model (deflated 48%)
  adding: my_t5_model/model.safetensors (deflated 10%)


In [ ]:
from google.colab import files
files.download('my_t5_model.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>